In [ ]:
from tqdm.notebook import tqdm, trange
from matplotlib.pyplot import imshow, subplots
import cv2
import numpy as np
import glob
import os
import pandas as pd
from skimage.measure import compare_ssim as ssim
from skimage.measure import compare_mse as mse
from skimage.measure import compare_psnr as psnr
from PIL import Image
from core.utils import ZipReader

In [1]:
def quantify_results(Method, direc, SaveName, OrigFrameFolder, OrigAnnotFolder, videoNames, options, zipped, Dil=8):
        #for cross change all ellipse to cross
        columnNames=["videoName", "Method", "PSNRavg","PSNRstd", "SSIMavg","SSIMstd", "MSEavg","MSEstd","PSNRCropavg","PSNRCropstd", "SSIMCropavg","SSIMCropstd", "SSIMCropFullavg","SSIMCropFullstd", "MSECropavg","MSECropstd"]
        df = pd.DataFrame(columns=columnNames)            
        
        for videoName in tqdm(videoNames):
                print(videoName)
                MaskFile=OrigAnnotFolder+videoName
                TransFrameFiles=sorted(glob.glob(direc+videoName+options+"/*"))
                OrigFrameFile=OrigFrameFolder+videoName
                PSNR=[]
                MSE=[]
                SSIM=[]
                PSNRCrop=[]
                MSECrop=[]
                SSIMCrop=[]
                SSIMCropFull=[]
                cnt=0
                adder=0
                for TransFrameFile in TransFrameFiles:
                        cnt=cnt+1
                        numStr=int(''.join(filter(str.isdigit, os.path.basename(TransFrameFile))))
                        if cnt==1 and numStr==1: adder=1
                        else: adder=0
                        TransName=(str(numStr-adder)).zfill(5)+".jpg"
                        TransImage= np.array(Image.open(TransFrameFile))
                        if zipped: MaskImage = ZipReader.imread(MaskFile+".zip",TransName)
                        else: MaskImage = Image.open(MaskFile.replace("Frames","Annotations")+"/"+TransName)
                        MaskImage = np.array(MaskImage.convert('L'))
                        MaskImage = np.array(MaskImage > 199).astype(np.uint8) #Rema:from 0 to 199 changes to binary better
                        MaskImage = cv2.dilate(MaskImage, cv2.getStructuringElement(
                                cv2.MORPH_ELLIPSE, (Dil, Dil)), iterations=1) #Rema:Dilate only 1 iteration
                        if np.size(np.where(MaskImage==1))>=30:
                                if zipped: OrigImage=np.array(ZipReader.imread(OrigFrameFile+".zip",TransName))
                                else: OrigImage=np.array(Image.open(OrigFrameFile+"/"+TransName))
                                wm, hm = MaskImage.shape
                                MaskImageRGB = np.empty((wm, hm, 3), dtype=np.uint8)
                                MaskImageRGB[:, :, 2] =  MaskImageRGB[:, :, 1] =  MaskImageRGB[:, :, 0] =  MaskImage

                                TransImageFull=TransImage*MaskImageRGB

                                PSNR.append(psnr(OrigImage,TransImageFull))
                                MSE.append(mse(OrigImage,TransImageFull))
                                SSIM.append(ssim(OrigImage,TransImageFull,multichannel=True))

                                TransImageCrop=TransImage[MaskImage!=0]
                                OrigImageCrop=OrigImage[MaskImage!=0]

                                PSNRCrop.append(psnr(OrigImageCrop,TransImageCrop))
                                MSECrop.append(mse(OrigImageCrop,TransImageCrop))

                                #ssim with full gives mmsim, S
                                S=ssim(OrigImage,TransImage,multichannel=True,full=True)
                                S=S[1]
                                S=np.mean(S,axis=2)
                                Scrop=S[MaskImage!=0]
                                Scrop_mean=np.mean(Scrop,axis=0)
                                SSIMCropFull.append(Scrop_mean)

                                SSIMCrop.append(ssim(OrigImageCrop,TransImageCrop,multichannel=True))


                PSNRavg=np.mean(PSNR)
                PSNRstd=np.std(PSNR)

                MSEavg=np.mean(MSE)
                MSEstd=np.std(MSE)

                SSIMavg=np.mean(SSIM)
                SSIMstd=np.std(SSIM)

                PSNRCropavg=np.mean(PSNRCrop)
                PSNRCropstd=np.std(PSNRCrop)

                MSECropavg=np.mean(MSECrop)
                MSECropstd=np.std(MSECrop)

                SSIMCropavg=np.mean(SSIMCrop)
                SSIMCropstd=np.std(SSIMCrop)

                SSIMCropFullavg=np.mean(SSIMCropFull)
                SSIMCropFullstd=np.std(SSIMCropFull)

                df_temp=pd.DataFrame([[videoName, Method,
                                PSNRavg, PSNRstd, SSIMavg,SSIMstd, MSEavg,MSEstd,
                                PSNRCropavg, PSNRCropstd, SSIMCropavg, SSIMCropstd, 
                                SSIMCropFullavg, SSIMCropFullstd, MSECropavg, MSECropstd]],columns=columnNames)
                # df=df.append(df_temp) # old pandas version
                df = pd.concat([df, df_temp]) # pandas higher than 2.0

        df.to_csv(SaveName) 
        print("Saved results to",SaveName)

# Examples

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Set dilation factor for the masks
Dil = 8

# Directory containing zipped test videos
zipped = True
OrigFrameFolder = "datasets/EndoSTTN_dataset/JPEGImages/"
OrigAnnotFolder = "datasets/EndoSTTN_dataset/Annotations/"

# videos to be tested
videoNames=[f'hyperK_{str(i).zfill(3)}' for i in range(343, 374)]

# Directory containing inpainted results
inpainted_dir = "results/"
options = "/overlaid/shifted/frameresult/"

# List of directories containing inpainted results using different trained models
# NOTE: The inpainted results should be inpainted using shifted masks (i.e. using --shifted with test.py)
inpainted_subdirs = ["Inpainted_pretrained/gen_00009/full_video/",
           "Inpainted_pretrained/gen_00009/single_frame/"]

# Directory to save your results
SaveDir = "Evaluation_Results/"

# For each trained model (method)
for inpainted_subdir in  inpainted_subdirs:
    print("Processing", inpainted_subdir)
    os.makedirs(os.path.join(SaveDir, inpainted_subdir), exist_ok=True)
    direc=inpainted_dir+inpainted_subdir+"/"
    SaveName=SaveDir+inpainted_subdir+".csv"
    quantify_results(inpainted_subdir, direc, SaveName, OrigFrameFolder, OrigAnnotFolder, videoNames, options, zipped, Dil)
    

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Set dilation factor for the masks
Dil = 8

# Directory containing test videos
zipped = False
OrigFrameFolder = "datasets/"
OrigAnnotFolder = "datasets/"

# videos to be tested
videoNames=['ExampleVideo1_Frames']

# Directory containing inpainted results
inpainted_dir = "results/"
options = "/overlaid/shifted/frameresult/"

# List of directories containing inpainted results using different trained models
# NOTE: The inpainted results should be inpainted using shifted masks (i.e. using --shifted with test.py)
inpainted_subdirs = ["Inpainted_pretrained/gen_00009/full_video/",
           "Inpainted_pretrained/gen_00009/single_frame/"]

# Directory to save your results
SaveDir = "Evaluation_Results_NotInMainDataset/"

# For each trained model (method)
for inpainted_subdir in  inpainted_subdirs:
    print("Processing", inpainted_subdir)
    os.makedirs(os.path.join(SaveDir, inpainted_subdir), exist_ok=True)
    direc=inpainted_dir+inpainted_subdir+"/"
    SaveName=SaveDir+inpainted_subdir+".csv"
    quantify_results(inpainted_subdir, direc, SaveName, OrigFrameFolder, OrigAnnotFolder, videoNames, options, zipped, Dil)
    